<a href="https://colab.research.google.com/github/DataScientistTX/NLP_tabular/blob/main/examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with Data Notebook (Embedded Tables)

In this notebook we walk you through an advanced RAG use case - parsing a complex document with embedded tables - and how to handle that in LlamaIndex.

We compare our approach against a "naive" RAG stack (using top-k retrieval with a fixed chunk size).



In [ ]:
#!pip install llama-index llama-hub pypdf --upgrade
#!apt install ghostscript python3-tk
#!pip install camelot-py pymupdf frontend ghostscript
!wget "https://www.dropbox.com/scl/fi/waoz9bo9yiemnhnqvu0cc/billionaires_page.pdf?rlkey=4i08msa7zr1lpnuq2y1vs2xgw&dl=1" -O billionaires_page.pdf

In [ ]:
# setup OpenAI
import openai

openai.api_key = ""

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Complex Document (with Embedded Tables)

In this setting we walk through a document that has an embedded table inside of it.

In [ ]:
import camelot
from llama_index import Document, SummaryIndex

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index import VectorStoreIndex, ServiceContext, LLMPredictor
from llama_index.query_engine import PandasQueryEngine, RetrieverQueryEngine
from llama_index.retrievers import RecursiveRetriever
from llama_index.schema import IndexNode
from llama_index.llms import OpenAI

from llama_hub.file.pymu_pdf.base import PyMuPDFReader
from pathlib import Path
from typing import List

### Parse out Table, build Pandas Query Engine

In [ ]:
# initialize PDF reader
reader = PyMuPDFReader()

In [ ]:
file_path = "billionaires_page.pdf"
docs = reader.load(file_path=file_path)

In [ ]:
# use camelot to parse tables
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        table_df = table_list[0].df
        table_df = (
            table_df.rename(columns=table_df.iloc[0])
            .drop(table_df.index[0])
            .reset_index(drop=True)
        )
        table_dfs.append(table_df)
    return table_dfs

In [ ]:
table_dfs = get_tables(file_path, pages=[3, 25])

In [ ]:
# shows list of top billionaires in 2023
table_dfs[0]

In [ ]:
# shows list of top billionaires
table_dfs[1]

In [ ]:
# define query engines over these tables
df_query_engines = [PandasQueryEngine(table_df) for table_df in table_dfs]

In [ ]:
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

In [ ]:
response = df_query_engines[1].query("How many billionaires were there in 2009?")
print(str(response))

### Build Recursive Retriever

We define a top-level vector index that does top-k lookup over a set of Nodes. We define two special nodes (`IndexNode` objects) linking to each of these tables.

We define a `RecursiveRetriever` object to recursively retrieve/query nodes. We then put this in our `RetrieverQueryEngine` along with a `ResponseSynthesizer` to synthesize a response.

We pass in mappings from id to retriever and id to query engine. We then pass in a root id representing the retriever we query first.

In [ ]:
llm = OpenAI(temperature=0, model="gpt-4")

service_context = ServiceContext.from_defaults(
    llm=llm,
)

In [ ]:
doc_nodes = service_context.node_parser.get_nodes_from_documents(docs)

In [ ]:
# define index nodes
summaries = [
    "This node provides information about the world's richest billionaires in 2023",
    "This node provides information on the number of billionaires and their combined net worth from 2000 to 2023.",
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [ ]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(doc_nodes + df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(
    # service_context=service_context,
    response_mode="compact"
)

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

### Define Baseline Retriever

We also define a baseline retriever that does top-k lookup over the raw document.

In [ ]:
# baseline vector index (that doesn't include the extra df nodes).
# used to benchmark
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

### Compare Results

We compare results between the recursive retriever vs. baseline retriever.

In [ ]:
response = query_engine.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

In [ ]:
response.source_nodes[0].node.get_content()

In [ ]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)
print(str(response))

In [ ]:
print(response.source_nodes[1].node.get_content())

In [ ]:
response = query_engine.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

In [ ]:
response = vector_query_engine0.query(
    "What is the average age of top 5 billionaires in 2023? Make sure age is a float."
)
print(str(response))

Of course, just like the baseline vector query engine, the recursive retriever can answer semantic queries over the article as well.

In [ ]:
response = query_engine.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

In [ ]:
response = vector_query_engine0.query(
    "How is wealth accounted for in recipients if the billionaire is deceased?"
)
print(str(response))

## Extended Use Case: Parsing Tesla 10Qs

Here we deal with an even messier document format - parsing tables within a 10K.

We use the Unstructured library to help us extract tables.

The tables aren't perfectly formatted, but work well enough.

In [ ]:
!pip install unstructured

### Extract Elements

In [ ]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

In [ ]:
!wget "https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1" -O tesla_2021_10k.htm

In [ ]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

reader = FlatReader()
docs_2021 = reader.load_data(Path("tesla_2021_10k.htm"))

In [ ]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser()

In [ ]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

In [ ]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [ ]:
example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][
    20
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
)

### Build Recursive Retriever

In [ ]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)

# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021, service_context=service_context)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

In [ ]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_2021,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

### Run Some Queries

In [ ]:
response = query_engine.query("What was the revenue in 2020?")
print(str(response))

In [ ]:
# compare against the baseline retriever
response = vector_query_engine.query("What was the revenue in 2020?")
print(str(response))